In [ ]:
%%capture
import requests
import numpy as np
%run DocumentSimilarity.ipynb

In [ ]:
KEY = "RqvYPz6UI8nsX5S21Kfpuke4xQ0lAwaD"

In [104]:
def GetResponse(params):
    headers = {"Authorization": f"Bearer {KEY}"}
    try:
        response = requests.get(f"https://api.core.ac.uk/v3/search/works{params}", headers=headers)
        if response.status_code != 200:
            return None    
        return response.json()
    except requests.exceptions.RequestException as e:
        print(e)
        return None

In [ ]:
def PrintKeys(d):
    print("Keys:")
    for key in d.keys():
        print(f"\t{key}")

In [ ]:
response = GetResponse("?limit=100&offset=0")

In [ ]:
if response != None:
    print(f"Number of Results: {len(response["results"])}")
    PrintKeys(response)
    print(response["totalHits"])

    print("Works:")
    for result in response["results"]:
        print(f"\"{result["title"][:50]}\"")
else:
    print("No Response")

In [ ]:
%%time
similarities = []
for result in response["results"]:
    if "language" in result and result["language"]["name"] != "English":
        similarities.append(0)
        continue
    if "abstract" not in result or not result["abstract"]:
        similarities.append(0)
        continue
    label, similarity = classifyText(result["abstract"])
    if label == 0:
        similarity = 1 - similarity
    similarities.append(similarity)

In [ ]:
sortedIndices = np.argsort(similarities)
for index in sortedIndices:
    print(index, similarities[index])

In [ ]:
print(response["results"][560])

In [112]:
def GetAllWorks():
    works = []
    offset = 0
    while True:
        response = GetResponse(f"?limit=10000&offset={offset}&q=title:*")
        if response is None or len(response["results"]) == 0:
            break
        offset += len(response["results"])
        works = [*response["results"], *works]
    return works
works = GetAllWorks()

In [113]:
len(works)

0

In [103]:
import json
import pandas

def query_api(url_fragment, query,is_scroll=False, limit=100, scrollId=None):
    api_endpoint = "https://api.core.ac.uk/v3/"
    headers={"Authorization":"Bearer "+KEY}
    query = {"q":query, "limit":limit}
    if not is_scroll:
        response = requests.post(f"{api_endpoint}{url_fragment}",data = json.dumps(query), headers=headers)
    elif not scrollId:
        query["scroll"]="true"
        response = requests.post(f"{api_endpoint}{url_fragment}",data = json.dumps(query),headers=headers)
    else:
        query["scrollId"]=scrollId
        response = requests.post(f"{api_endpoint}{url_fragment}",data = json.dumps(query),headers=headers)
    if response.status_code ==200:
        return response.json(), response.elapsed.total_seconds()
    else:
        print(f"Error code {response.status_code}, {response.content}")

def scroll(search_url, query, extract_info_callback=None):
    allresults = []
    count = 0
    scrollId=None
    while True:
        result, elapsed =query_api(search_url, query, is_scroll=True, scrollId=scrollId)
        scrollId=result["scrollId"]
        totalhits = result["totalHits"]
        result_size = len(result["results"])
        if result_size==0:
            break
        for hit in result["results"]:
            if extract_info_callback:
              allresults.append(extract_info_callback(hit))
            else:
              allresults.append(hit)
        count+=result_size
        print(f"{count}/{totalhits} {elapsed}s")
    return allresults

uk_data_providers_raw = scroll("search/data-providers", "location.countryCode:gb")
uk_data_providers = pandas.DataFrame(uk_data_providers_raw)
uk_data_providers

94/1008 1.416769s
192/1008 0.863104s
Error code 429, b''


TypeError: cannot unpack non-iterable NoneType object